# Introduction (draft)

This notebook assumes no prior knowledge of the concepts required for the albedos calc.

## MCD43A1

### Subdatasets

These SDSs are BRDF_Albedo_Parameters (three model parameters representing fiso, fvol, fgeo for the RossThickLiSparseReciprocal BRDF model that are computed for bands 1-7 and three broad bands), and BRDF_Albedo_Band_Mandatory_Quality* (containing 1 byte  of quality flags).
```
The MODIS BRDF/Albedo/NBAR product (MCD43) is led by Professor Crystal Schaaf at UMass Boston. 
* [User Guide](https://www.umb.edu/spectralmass/terra_aqua_modis/modis_brdf_albedo_product_mcd43) 
for the complete MCD43 User Guide. 
* [Albedo (HDF) source code](proc/actual_albedo_tool/actual_albedo_tool.tar.gz) 
for computing black, white, and blue sky albedo from the BRDF parameters (MCD43A1) is available at the lab's [ftp site](ftp://rsftp.eeos.umb.edu/data01/Website/actual_albedo_tool.tar.gz). 
* [source code readme](proc/actual_albedo_tool/albedo/readme)
I implemented the albedo algorithms in Python using the refactored equations given in the 
```

### Terms

**BRDF:** Bidirectional Reflectance Distribution Function           
...

## Retrieve daily BRDF parameters 

From AppEEARS in this case... Details coming soon...

**Request the 2018 BRDF Parameters**: [data/MCD43A1-Florida-2018-request.json](data/MCD43A1-Florida-2018-request.json)

In [9]:
import json
with open("data/MCD43A1-Florida-2018-request.json", "r") as f:
    d = {k:v for k,v in json.load(f).items() if k!="params"}
    print(json.dumps(d, indent=2))

{
  "error": null,
  "status": "processing",
  "created": "2019-06-21T17:00:02.310429",
  "task_id": "f9ecaf4c-4eee-4236-beee-50fcfeb50963",
  "updated": "2019-06-21T18:07:07.768944",
  "user_id": "jjmcnelis@outlook.com",
  "retry_at": null,
  "task_name": "MCD43A1_Florida_2018",
  "task_type": "area",
  "api_version": null,
  "svc_version": "2.23",
  "web_version": "2.23",
  "expires_on": "2019-07-21T18:07:07.768944",
  "attempts": 1
}




### Not data files 
```
MCD43A1-Florida-2018-MCD43A1-006-metadata.xml 	ISO 19115 Metadata
MCD43A1-Florida-2018-granule-list.txt 			URLs for all source data
README.txt 									   Instructions/details about the request
MCD43A1-Florida-2018-request.json 				JSON to create a new request
```

The json [*MCD43A1-Florida-2018-request.json*](data/MCD43A1-Florida-2018-request.json) can be uploaded to AppEEARS to order an identical subset to the one used in the next notebook.

## Data quality

Reference: https://lpdaac.usgs.gov/products/mcd43a1v006/

Fortunately, MCD43 quality flags are as simple as they come (for MODIS):
```
Value 	Description
0 	    Processed; good quality (full BRDF inversions)
1 	    Processed; see other QA (magnitude BRDF inversions)
2 	    Processed; good quality (full BRDF inversions); Band 6 filled; dead or noisy detectors
3 	    Processed; see other QA (magnitude BRDF inversions); Band 6 filled; dead or noisy detectors
255 	Fill Value
```

### Ancillary files

<table>
    <col width="130">
    <col width="80">
    <thead>
        <tr>
            <th style="text-align: left;">File</th>
            <th style="text-align: left;">Description</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><i>MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-<b>BAND</b>-Statistics-QA.csv</i></td>
            <td>Statistics for quality layers</td>
        </tr>
         <tr>
            <td><i>MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-<b>BAND</b>-lookup.csv</i></td>
            <td>Lookup for quality bits</td>
        </tr>
    </tbody>
</table>

```
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band1-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band1-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band2-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band2-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band3-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band3-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band4-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band4-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band5-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band5-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band6-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band6-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band7-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-Band7-lookup.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-nir-Statistics-QA.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-nir-lookup.csv			
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-shortwave-Statistics-QA.csv
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-shortwave-lookup.csv	
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-vis-Statistics-QA.csv		
MCD43A1-006-BRDF-Albedo-Band-Mandatory-Quality-vis-lookup.csv	
```

### Next


## Albedo

### User Guide
https://www.umb.edu/spectralmass/terra_aqua_modis/v006/mcd43a1_brdif_albedo_model_parameters_product

### Black sky albedo 

#### Constants

<table width="50%">
<tr>
<th>Term</td>
<th>Isotropic (iso)</th>
<th>RossThick (vol)</th>
<th>LiSparseR (geo)</th>
</tr>
<tr>
<td>g<sub>0</sub></td>
<td>&nbsp;1.0</td>
<td>-0.007574</td>
<td>-1.284909</td>
</tr>
<tr>
<td>g<sub>1</sub></td>
<td>&nbsp;0.0</td>
<td>-0.070987</td>
<td>-0.166314</td>
</tr>
<tr>
<td>g<sub>2</sub></td>
<td>&nbsp;0.0</td>
<td>&nbsp; &nbsp;0.307588</td>
<td>&nbsp; &nbsp;0.041840</td>
</tr>
</table>


#### Equation
![bsa](https://www.umb.edu/editor_uploads/images/csm_eeos/images/eqn2small.jpg)

### White sky albedo

#### Constants

<table width="50%">
<tr>
<th>Term</td>
<th>Isotropic (iso)</th>
<th>RossThick (vol)</th>
<th>LiSparseR (geo)</th>
</tr>
<tr>
<td>White-sky<br>
integral <i>g</i></td>
<td>&nbsp;1.0</td>
<td>&nbsp;0.189184</td>
<td>-1.377622</td>
</tr>
</table>



#### Equation
![wsa](https://www.umb.edu/editor_uploads/images/csm_eeos/images/eqn3small.jpg)


### Blue sky Albedo

Continue...

## Calculate solar angles at local noon

These calculations were harvested from: 
http://holbert.faculty.asu.edu/eee463/SolarCalcs.pdf

The ORNL DAAC uses an executable that is probably ten years old and we don't have the source code! So, the longest step in this process was determining which equations would yield the same result as that executable. Very frustrating. Nonetheless, I'm very confident in the result. 

Import `math.radians` and `math.cos` for the calculation:

In [4]:
from math import radians, cos

### Declination angle

https://sciencing.com/calculate-suns-declination-6904335.html

In [7]:
doy = 166                  # day of the year    
diy = 365                  # number of days in the year
deg_rot_per_day = 360/diy  # degree of rotation per day
abs_max_decl = 23.44       # absolute |maximum declination angle|

deg_rot_per_day

0.9863013698630136

Get cosine of x and multiple it by the negative of the max lat (axial tilt, -23.44) to get solar declination in degrees:

In [8]:
decl = cos(radians((doy+10)*deg_rot_per_day))*-abs_max_decl
decl

23.293419845424015

### Solar zenith angle
The sum of the solar zenith angle and the solar altitude angle equal 90 degrees. In the northern hemisphere, get the solar altitude angle by subtracting latitude from 90 and summing with the declination angle. 

Compute SZA at 35 degrees latitude on day of the year 166. First get the declination:

In [9]:
lat, doy, ndoy = 35., 166., 365
decl = cos(radians((doy+10)*(360/ndoy)))*-23.45

decl

23.303357311228375

Now get solar altitude angle:

In [44]:
solar_alt = 90. - lat + decl

solar_alt

78.30335731122838

Now subtract solar altitude from 90 to get the solar zenith angle:

In [45]:
solar_zen = 90. - solar_alt

solar_zen

11.696642688771618

As a function:

In [46]:
def get_solar_zenith(latitude, doy, ndoy=365):
    """ """
    declination = cos(radians((doy+10)*(360/ndoy)))*-23.45
    solar_altitude = 90 - latitude + declination
    solar_zenith = 90 - solar_altitude
    return(solar_zenith)
    
get_solar_zenith(35, 166)

11.696642688771618

The result matches the output from the solar zenith calculator used by ORNL DAAC:
```shell
[jnd@daacmodis1 plug_1]$ ./local_szn.1.exe 35 166
lat 35.0000000 day 166 lzsn 11.696643
```